<a href="https://colab.research.google.com/github/aeleraqi/Vid2TXT/blob/main/Vid2TXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install Dependencies and Import Libraries

In [16]:
# Install necessary libraries
!pip install pydub
!pip install SpeechRecognition
!pip install python-docx
!pip install mercury
!pip install pandas
!pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [17]:
# Import necessary libraries
import os
import io
import csv
from datetime import datetime, timedelta
import pandas as pd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import speech_recognition as sr
from docx import Document
import mercury as mr

###Speech Recognition Function

In [18]:
def recognize_speech_google(audio_file):
    """
    Recognize speech from an audio file using Google Speech Recognition.
    """
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)

    try:
        # Recognize the audio, assuming the language is Arabic
        text = recognizer.recognize_google(audio, language='ar-EG')
    except sr.UnknownValueError:
        text = "(Could not understand)"
    except sr.RequestError as e:
        text = f"(Error: {e})"

    return text

###Main Function to Process Audio and Save Output to DOCX

In [19]:
def main(audio_file, output_docx):
    """
    Process an audio file to recognize speech and save results to a DOCX file.
    """
    # Load the audio file
    audio = AudioSegment.from_file(audio_file)

    # Split audio into chunks based on silence
    chunks = split_on_silence(audio, min_silence_len=500, silence_thresh=-40)

    output_data = []
    start_time = datetime.now()

    # Process each chunk
    for chunk in chunks:
        # Export each chunk to a temporary wav file
        chunk.export("temp.wav", format="wav")

        # Recognize speech from the exported chunk
        text = recognize_speech_google("temp.wav")

        # Calculate timestamp for the chunk
        duration = timedelta(seconds=chunk.duration_seconds)
        end_time = start_time + duration
        timestamp = end_time.strftime("%H:%M:%S")

        # Append timestamp and recognized text to the output data
        output_data.append((timestamp, text))

        # Update start time for the next chunk
        start_time = end_time

    # Create a new Word document
    doc = Document()

    # Add a title to the document
    doc.add_heading('Speech Recognition Output', 0)

    # Create a table with two columns (Timestamp and Text)
    table = doc.add_table(rows=1, cols=2)

    # Set the headers
    hdr_cells = table.rows[0].cells
    hdr_cells[0].text = 'Timestamp'
    hdr_cells[1].text = 'Text'

    # Add the data to the table
    for timestamp, text in output_data:
        row_cells = table.add_row().cells
        row_cells[0].text = timestamp
        row_cells[1].text = text

    # Save the document
    doc.save(output_docx)


###Main Script Execution

In [20]:
if __name__ == "__main__":
    # Upload the audio file using Google Colab's file upload widget
    print("Please upload your audio file:")
    uploaded = files.upload()

    # Get the file name from the uploaded files
    audio_file = list(uploaded.keys())[0]

    # Specify the output DOCX file name
    output_docx = "output.docx"

    # Call the main function with the uploaded audio file
    main(audio_file, output_docx)


Please upload your audio file:


Saving test.mp3 to test.mp3
